In [91]:
#import all libraries
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [92]:
#get and split low_res data
height = 299
width = 299
lr_batch_size = 4

lr_train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2)

lr_train_generator = lr_train_datagen.flow_from_directory(
    'C:/Users/stolk/Desktop/Workspaces/ML/malaria/low_res/data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=lr_batch_size,
    class_mode='binary',
    subset='training')
lr_validation_generator = lr_train_datagen.flow_from_directory(
    'C:/Users/stolk/Desktop/Workspaces/ML/malaria/low_res/data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=lr_batch_size,
    class_mode='binary',
    subset='validation')

lr_epochs = 1
lr_steps = 22048 // 1
lr_val_steps = 5510 // 1

print(lr_steps)
print(lr_val_steps)

Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.
22048
5510


In [93]:
#build inceptionv3 algorithm and compile
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)

base_model = InceptionV3(weights=None, include_top=False, input_tensor=Input(shape=input_shape))

out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(1024, activation='relu')(out)
predictions = Dense(1, activation='sigmoid')(out)

model = Model(inputs=base_model.input, outputs=predictions)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
    



LR = 0.0001
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

In [94]:
#train to fit low_res data
history = model.fit(
    lr_train_generator,
    epochs=lr_epochs,
    steps_per_epoch=lr_steps // lr_batch_size,
    validation_data=lr_validation_generator,
    validation_steps=lr_val_steps // lr_batch_size
)
model.trainable = False

5512/5512 [==============================] - 556s 99ms/step - loss: 0.3560 - accuracy: 0.8432 - f1_m: 0.7846 - precision_m: 0.7954 - recall_m: 0.8153 - val_loss: 0.1643 - val_accuracy: 0.9424 - val_f1_m: 0.8642 - val_precision_m: 0.8848 - val_recall_m: 0.8637


In [95]:
#check accuracy
results = model.evaluate(
    lr_validation_generator,
    steps=lr_val_steps // lr_batch_size,
)

print("loss: " + str(results[0]))
print("acc: " + str(results[1]))
print("f1: " + str(results[2]))
print("pre: " + str(results[3]))
print("rec: " + str(results[4]))

1377/1377 [==============================] - 41s 29ms/step - loss: 0.1633 - accuracy: 0.9439 - f1_m: 0.8759 - precision_m: 0.8946 - recall_m: 0.8765
loss: 0.16329272091388702
acc: 0.9438998103141785
f1: 0.8759000301361084
pre: 0.8946380019187927
rec: 0.8765432238578796


In [39]:
# full none
# 
# acc: 0.9438998103141785
# f1: 0.8759000301361084
# pre: 0.8946380019187927
# rec: 0.8765432238578796

# full img
# 
# acc: 0.9509803652763367
# f1: 0.8894315361976624
# pre: 0.8993586301803589
# rec: 0.8959693908691406

# 1/2 none
# acc: 0.9411337375640869
# f1: 0.882910430431366
# pre: 0.8679748177528381
# rec: 0.918483555316925

# 1/2 img
# acc: 0.9571220874786377
# f1: 0.9032805562019348
# pre: 0.9016472697257996
# rec: 0.9204215407371521

# 1/4 none
#
# acc: 0.9004360437393188
# f1: 0.8495294451713562
# pre: 0.8192830681800842
# rec: 0.9140018820762634

# 1/4 img
#
# acc: 0.9549418687820435
# f1: 0.8903377652168274
# pre: 0.8943797945976257
# rec: 0.9011628031730652

# 1/8 none
# 
# acc: 0.694767415523529
# f1: 0.47411394119262695
# pre: 0.6143410801887512
# rec: 0.413759708404541

# 1/8 img
# 
# acc: 0.9534883499145508
# f1: 0.8955152034759521
# pre: 0.9040697813034058
# rec: 0.9040697813034058

# 1/16 none
# 
# acc: 0.5174418687820435
# f1: 0.64418625831604
# pre: 0.5174418687820435
# rec: 0.9534883499145508

# 1/16 img
# 
# acc: 0.9186046719551086
# f1: 0.8664452433586121
# pre: 0.9089148044586182
# rec: 0.846899151802063




In [ ]:
model_location = 'D:\.Downloads\models\\allinone\modelweights.h5'
base_model.save_weights(model_location)